In [1]:
import pandas as pd 
from utils import read_df, save_df_parquet, sample_series
import ast
import nltk

In [2]:
df: pd.DataFrame = read_df('dados_tokenizados.csv')

In [3]:
df.sample(3)

,id,canal,texto_original,temas,texto_limpo,tokens,tokens_clean
864,1282,Participe+,Meta 4 - Iniciativa B - Revisar - Incluindo no...,['Saúde'],meta 4 - iniciativa b - revisar - incluindo no...,"['meta', '4', '-', 'iniciativa', 'b', '-', 're...","['b', 'revisar', 'incluindo', 'atendimento', '..."
82,83,Audiência Regional - Parelheiros,Boa tarde a todos. É um prazer estar participa...,[],boa tarde a todos. é um prazer estar participa...,"['boa', 'tarde', 'a', 'todos', '.', 'é', 'um',...","['boa', 'tarde', 'todos', 'prazer', 'participa..."
1277,1978,Participe+,Meta 37 - Iniciativa O - Revisar - Considerand...,['Transportes e Mobilidade'],meta 37 - iniciativa o - revisar - considerand...,"['meta', '37', '-', 'iniciativa', 'o', '-', 'r...","['revisar', 'considerando', 'prefeitura', 'ati..."


Para construir a bag of words vou precisar apenas dos tokens e da classificacao.

Terei que fazer o explode na classificação - pois um mesmo texto pode ter mais de um tema

In [4]:
df = df[['tokens_clean', 'temas']]

In [5]:
df.dtypes

tokens_clean    object
temas           object
dtype: object

In [6]:
df.head()

,tokens_clean,temas
0,"['boa', 'tarde', 'todas', 'todos', 'nome', 'fr...",['Meio Ambiente']
1,"['oi', 'mario', 'sergio', 'bortolotto', 'favor...",['Urbanismo e Licenciamento']
2,"['entao', 'boa', 'tarde', 'todos', 'todas', 'c...","['Zeladoria Urbana e melhorias de bairro', 'De..."
3,"['obrigada', 'agradeco', 'oportunidade', 'faco...","['Zeladoria Urbana e melhorias de bairro', 'De..."
4,"['bom', 'dia', 'todos', 'nome', 'maria', 'luiz...",['Zeladoria Urbana e melhorias de bairro']


In [7]:
def parse_lst_column(lst_str)->list[str]:

    if type(lst_str) is not str:
        #bizarro - mas alguns registros estao como listas ou como arrays mesmo
        lst_str = f"[{', '.join(lst_str)}]"

    lst_str = lst_str.lstrip('array')

    return ast.literal_eval(lst_str)
    

In [8]:
df['temas'] = df['temas'].apply(parse_lst_column)
df['tokens_clean'] = df['tokens_clean'].apply(parse_lst_column)

In [9]:
df.sample(3)

,tokens_clean,temas
92,"[agradeco, continuo, insistindo, persistindo, ...",[Desenvolvimento Econômico e Trabalho]
586,"[revisar, principalmente, periferia, asfalto, ...",[Zeladoria Urbana e melhorias de bairro]
1531,"[ao, alteracao, uuso, ocupacao, solo, reunimos...",[Urbanismo e Licenciamento]


In [10]:
df = df.explode('temas')
df.rename({'temas' : 'tema'}, axis=1, inplace=True)

df.sample(3)

,tokens_clean,tema
215,"[nome, lucas, moro, aqui, regiao, campo, limpo...",Transportes e Mobilidade
2180,"[documento, protocolado, acompanhamento, elabo...",Habitação
16,"[professor, aposentado, trabalho, area, agua, ...",Zeladoria Urbana e melhorias de bairro


In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df['id'] = df.index

In [13]:
df.head()

,tokens_clean,tema,id
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
df['corpora'] = df['tokens_clean'].apply(lambda x: ' '.join(x))

In [16]:
df.head()

,tokens_clean,tema,id,corpora
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0,boa tarde todas todos nome francisco apelido c...
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1,oi mario sergio bortolotto favor responde enco...
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2,entao boa tarde todos todas comprimento secret...
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3,entao boa tarde todos todas comprimento secret...
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4,obrigada agradeco oportunidade faco parte rede...


In [17]:
df = df[['id', 'corpora', 'tema']]

In [18]:
df.head()

,id,corpora,tema
0,0,boa tarde todas todos nome francisco apelido c...,Meio Ambiente
1,1,oi mario sergio bortolotto favor responde enco...,Urbanismo e Licenciamento
2,2,entao boa tarde todos todas comprimento secret...,Zeladoria Urbana e melhorias de bairro
3,3,entao boa tarde todos todas comprimento secret...,Desenvolvimento Econômico e Trabalho
4,4,obrigada agradeco oportunidade faco parte rede...,Zeladoria Urbana e melhorias de bairro


In [19]:
vectorizer = CountVectorizer()

In [20]:
bow_matrix= vectorizer.fit_transform(df['corpora'])

In [21]:
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [22]:
bow_df.head()

,1000km,100km,10ampliar,10h,10km,119c10201210,11o,12e3,12milhoes,12o,...,zildo,zituo,zn,zogbi,zom,zona,zonas,zoneamento,zr1,zum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.shape

(3581, 3)

In [24]:
bow_df.shape

(3581, 16701)

In [25]:
bow_df['id'] = df['id']

In [26]:
bow_df['y'] = df['tema']

In [27]:
bow_df.head()

,1000km,100km,10ampliar,10h,10km,119c10201210,11o,12e3,12milhoes,12o,...,zituo,zn,zogbi,zom,zona,zonas,zoneamento,zr1,zum,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Meio Ambiente
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urbanismo e Licenciamento
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zeladoria Urbana e melhorias de bairro
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Desenvolvimento Econômico e Trabalho
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zeladoria Urbana e melhorias de bairro


In [28]:
save_df_parquet(bow_df, 'bag_of_words_df.parquet')

'../data/bag_of_words_df.parquet'